Importing libraries and scripts

In [ ]:
%matplotlib inline
import nbimport
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
import scipy.sparse.linalg as spla
from pyamg import smoothed_aggregation_solver, rootnode_solver
import scipy.sparse as spa
np.set_printoptions(linewidth=130)

In [ ]:
from fsmfuncs import *
from ibmfuncs import *
from gridfuncs import *

Numerical grid for the fluid

In [3]:
s1 = stretching(50, 0.033, 0.20, int(0.65/0.033), 16, 16, 0.04)[0]
s2 = stretching(50, 0.033, 0.20, int(0.65/0.033), 16, 16, 0.04)[0]
s3 = stretching(50, 0.033, 0.20, int(0.65/0.033), 16, 16, 0.04)[0]
s4 = stretching(256, 0.033, 0.20, int(0.65/0.033), 16, 16, 0.04)[0]

x1 = np.concatenate([-s2[::-1], s1[1:]])
x2 = np.concatenate([-s3[::-1], s4[1:]])+ (s2[-1]+s1[-1])
x = np.concatenate([x1[:-1], x2[1:]])

s = stretching(192, 0.033, 0.20, int(0.65/0.033), 16, 16, 0.04)[0]
y = np.concatenate([-s[::-1], s[1:]])

n, m = len(y)-1, len(x)-1

dy, dx = np.diff(y), np.diff(x)
dxmin = min(np.min(dx), np.min(dy))

# Pressure
yp, xp = 0.5*(y[1:] + y[:-1]), 0.5*(x[1:] + x[:-1])
dyp, dxp = np.diff(yp), np.diff(xp)
p = np.zeros( (n, m) )

# Horizontal velocity
yu, xu = yp, x[1:-1]
u = np.zeros( (n, m-1) )

# Vertical velocity
yv, xv = y[1:-1], xp
v = np.zeros( (n-1, m) )

Immersed boundary

In [ ]:
r_ = 0.5
l = int((2*np.pi*r_)/dxmin)
ang_ = 2*np.pi*np.arange(l)/l

xi = np.zeros(2*l)
eta = np.zeros(2*l)

# Coordinates of the first cylinder
xi[:l] = 0 + r_*np.cos(ang_)
eta[:l] = 0 + r_*np.sin(ang_)

# Coordinates of the second cylinder
xi[l:] = (s2[-1]+s1[-1]) + r_*np.cos(ang_)
eta[l:] = 0 + r_*np.sin(ang_)

ds = 2*np.pi*r_/l*np.ones(2*l)

# Velocity vectors for both cylinders
UB = np.zeros(4)
UBm1 = np.zeros(4)

# Position vector for both cylinders
XB = np.zeros(4)
XB[2]=xi[l]


Plot grid and immersed boundary

In [ ]:
plt.figure(figsize=(8,8))
X, Y = np.meshgrid(x, y)
plt.plot(X, Y, 'b-');
plt.plot(X.T, Y.T, 'b-');
plt.plot(xi[:l], eta[:l], 'ro-');
plt.plot(xi[l:], eta[l:], 'go-');
plt.axis('equal')

Boundary conditions + initial flow

In [6]:
uS, uN = np.ones(m-1), np.ones(m-1)
uE, uW = np.ones(n), np.ones(n)

vS, vN = np.zeros(m), np.zeros(m)
vE, vW = np.zeros(n-1), np.zeros(n-1)

u[:,:]=1
v[:,:]=0

Build matrices (I)

In [7]:
G, DuW, DuE, DvS, DvN = gradient(dxp, dyp)
R, iR = weight (dx, dy)
Mh, iMh = mass_hat (dxp, dyp)
Lh, Lux0, Lux1, Luy0, Luy1, Lvx0, Lvx1, Lvy0, Lvy1 = laplacian_hat(dx, dy, dxp, dyp)

L = Mh.dot(Lh.dot(iR))

M = Mh.dot(iR)
iM = R.dot(iMh)

iML = iM.dot(L)

In [ ]:
# Reynlods number
iRe = 1/200.0

# Time step
dt = 0.30 * min(dxmin**2/iRe, dxmin)

print(dt, dxmin**2/iRe, dxmin)

Build matrices (II)

In [9]:
A = (M/dt - 0.5*iRe*L).tocsr()
B = (M/dt + 0.5*iRe*L).tocsr()

BN = dt*iM + (0.5*iRe)*dt**2*iML.dot(iM) + (0.5*iRe)**2*dt**3*iML.dot(iML.dot(iM))


In [10]:
# Velocity flux
q = np.loadtxt('q-Re=200')
qast = q.copy()

UBast = UB.copy()

Num1, Nvm1 = advection_hat(dx, dy, dxp, dyp, iR.dot(q),  uS, uN, uW, uE, vS, vN, vW, vE)
Nu, Nv = Num1, Nvm1

GTBNG = G.T.dot(BN.dot(G))

 Dimensionless parameters

In [ ]:
# Density ratios
rho1 = 300 
rho2 = 300
# Volume
Vs = np.pi/4
# Spring constants
K1 = 1.57
K2 = 1.57

Build matrices (III)

In [11]:
IB = np.zeros((4,4))
IB[0,0]= Vs*rho1/dt
IB[1,1]=Vs*rho1/dt
IB[2,2]=Vs*rho2/dt
IB[3,3]=Vs*rho2/dt

iIB = np.zeros((4,4))
iIB[0,0]=dt/(Vs*rho1)
iIB[1,1]=dt/(Vs*rho1)
iIB[2,2]=dt/(Vs*rho2)
iIB[3,3]=dt/(Vs*rho2)

NB = - np.ones((4,4*l))
NB[0,:]= 0
NB[2,:]= 0
NB[1,:l]=np.zeros(l)
NB[1,2*l:4*l]= np.zeros(2*l)
NB[3,:3*l]=np.zeros(3*l)

Time loop

In [ ]:
nt = int(50/dt)
print("Performing", nt, "steps")

residuals = np.zeros(nt)

# Aerodynamic coefficients for both cylinders
CFx1 = np.zeros(nt)
CFy1 = np.zeros(nt)
CFx2 = np.zeros(nt)
CFy2 = np.zeros(nt)

# Velocity and position of the front cylinder
Vc1 = np.zeros(nt)
Yc1 = np.zeros(nt)

# Velocity and position of the rear cylinder
Vc2 = np.zeros(nt)
Yc2 = np.zeros(nt)

for k in range(nt):  
    
    # Computing regularization and interpolation operators
    Eh = interpolation_hat(xi, eta, ds, x, y, xp, yp, dx, dy, dxp, dyp)
    Hh = regularization_hat(xi, eta, ds, x, y, xp, yp, dx, dy, dxp, dyp)

    E = Eh.dot(iR)
    H = Mh.dot(Hh)
    
    dQ = (Vs/dt)*(UB-UBm1)
    
    UBast[0]= UB[0]
    UBast[1]= UB[1]+dt/(rho1*Vs)*dQ[1]-dt*K1*XB[1]
    UBast[2]= UB[2]
    UBast[3]= UB[3]+dt/(rho2*Vs)*dQ[3]-dt*K2*XB[3]
    
    ru = iRe*(Lux0.dot(uW) + Lux1.dot(uE) + Luy0.dot(uS) + Luy1.dot(uN)) - 1.5*Nu + 0.5*Num1
    rv = iRe*(Lvx0.dot(vW) + Lvx1.dot(vE) + Lvy0.dot(vS) + Lvy1.dot(vN)) - 1.5*Nv + 0.5*Nvm1 #-g 
    
    bc1 = Mh.dot(np.concatenate([ru, rv]))
    r1 = B.dot(q.ravel()) + bc1
   
    # Resloving the first linear equation
    mls = smoothed_aggregation_solver(A)
    accelerated_residuals = []
    qast= mls.solve( r1, tol=1e-8, accel='cg', residuals=accelerated_residuals)
    
    bc2 = - (DuW.dot(uW*dy) + DuE.dot(uE*dy) + DvS.dot(vS*dx) + DvN.dot(vN*dx))
    
    GTBNET = G.T.dot(BN.dot(E.T))
    EBNG = E.dot(BN.dot(G))
    X = E.dot(BN.dot(E.T)) + NB.T.dot(iIB.dot(NB))
    
    
    e = spa.hstack((GTBNG,GTBNET))
    j = spa.hstack((EBNG,X))
    Q = spa.vstack((e,j)).tocsr()
    
    a = np.concatenate((G.T.dot(qast.ravel())+bc2, E.dot(qast.ravel())+ NB.T.dot(UBast)))
    
    # Resolving the second linear equation
    mls1 = smoothed_aggregation_solver(Q)
    accelerated_residuals = []
    λ = mls1.solve(a, tol=1e-8, accel='cg', residuals=accelerated_residuals)
    
    # Projection step
    qp1 = qast - BN.dot(G.dot(λ[:n*m])) - BN.dot(E.T.dot(λ[n*m:]))
    UB1 = UBast - iIB.dot(NB.dot(λ[n*m:]))
    
    residuals[k] = la.norm(qp1-q)/(la.norm(dt*qp1)) 
    
    CFx1[k]=2*np.sum(λ[n*m:n*m+l])
    CFx2[k]=2*np.sum(λ[n*m+l:n*m+2*l])
    CFy1[k]=2*np.sum(λ[n*m+2*l:n*m+3*l])
    CFy2[k]=2*np.sum(λ[n*m+3*l:n*m+4*l])
    
    # Updating coordinates
    eta[:l] = eta[:l] + UB1[1]*dt
    eta[l:] = eta[l:] + UB1[3]*dt
    XB[1]= XB[1]+ UB1[1]*dt
    XB[3]= XB[3]+ UB1[3]*dt
    
    # Storage of the position and velocity of the front cylinder
    Yc1[k]= XB[1]
    Vc1[k]= UB[1]
    
    # Storage of the position and velocity of the rear cylinder
    Yc2[k]= XB[3]
    Vc2[k]= UB[3]
    
    
    if k%100==0:
        print(k, k*dt, residuals[k], CFy1[k],CFy2[k], XB[1], UB[1], XB[3], UB[3])
    
    q = qp1.copy()
    UBm1 = UB.copy()
    UB = UB1.copy()
    
    # Free exit condition 
    uE = uE - dt/dx[-1]*(uE - iR.dot(q)[:n*(m-1)].reshape((n, m-1))[:,-1])
    
    Num1, Nvm1 = Nu, Nv
    Nu, Nv = advection_hat(dx, dy, dxp, dyp, iR.dot(q), uS, uN, uW, uE, vS, vN, vW, vE)
    

Velocity and pressure

In [ ]:
iRq = iR.dot(q)
u, v = iRq[:n*(m-1)].reshape((n, m-1)), iRq[n*(m-1):].reshape((n-1, m))
p = λ[:n*m].reshape((n,m))

Display solution

In [ ]:
# Ploting velocity
x0, x1 = -2, 7
y0, y1 = -4.5, 4.5
plt.figure(figsize=(5.5*3,4))
plt.subplot(1,3,1)
plt.pcolormesh(xu, yu, u, shading='gouraud')
plt.plot(xi, eta, lw=1)
plt.xlim(x0, x1)
plt.ylim(y0, y1)
plt.colorbar()

plt.subplot(1,3,2)
plt.pcolormesh(xv, yv, v, shading='gouraud')
plt.plot(xi, eta, lw=1)
plt.xlim(x0, x1)
plt.ylim(y0, y1)
plt.colorbar()

# Ploting pressure
plt.subplot(1,3,3)
plt.pcolormesh(xp, yp, p, shading='gouraud')
plt.plot(xi, eta, lw=1)
plt.xlim(x0, x1)
plt.ylim(y0, y1)
plt.colorbar()

In [ ]:
# Ploting aerodynamic coefficients
plt.plot(np.arange(len(CFx1))*dt, CFx1, label='Cd1')
plt.plot(np.arange(len(CFx2))*dt, CFx2,'r',label='Cd2')
plt.plot(np.arange(len(CFy1))*dt, CFy1,'g', label='Cl1')
plt.plot(np.arange(len(CFy2))*dt, CFy2,'k',label='Cl2')
plt.xlim(0, 50)
plt.ylim(-0.8,5)
plt.xlabel('t')
plt.ylabel('Aerodynamic coefficients')
plt.legend()

In [ ]:
# Ploting the velocity of both cylinders
plt.plot(np.arange(len(CFy1))*dt, Vc1, 'r-', label='Front cylinder')
plt.plot(np.arange(len(CFy1))*dt, Vc2, 'b-', label='Rear cylinder')
plt.xlim(0,50)
plt.ylim(-0.01,0.01)
plt.xlabel('t')
plt.ylabel('Vc')
plt.legend()

In [ ]:
# Ploting the position of both cylinders
plt.plot(np.arange(len(CFy1))*dt, Yc1, 'r-', label='Front cylinder')
plt.plot(np.arange(len(CFy1))*dt, Yc2, 'b-', label='Rear cylinder')
plt.xlim(0,50)
plt.ylim(-0.01,0.01)
plt.xlabel('t')
plt.ylabel('Yc')
plt.legend()